In [4]:
import ee
import geemap
import matplotlib.pyplot as plt
import wxee

In [5]:
ee.Authenticate()


Successfully saved authorization token.


In [6]:
wxee.Initialize()

In [13]:
def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask).divide(10000)

In [15]:
# set system:time_start as a property if it doesn't exist
def set_time(image):
    return image.set({'system:time_start':image.date().millis()})

In [9]:
aoi = ee.Geometry.Polygon(
  [[[8.559718,  49.952662],
    [8.559718, 49.794677],
    [8.752842, 49.794677],
    [8.752842, 49.952662]]]
  )

dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      #.map(mask_s2_clouds)
      .map(set_time)
      .filterBounds(aoi)
      .filterDate('2017-04-01', '2023-10-27')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
      .select(['B4'])
)
dataset = dataset
